In [1]:
import cv2
import numpy as np
import tensorflow as tf
import pickle

In [4]:
MODEL_PATH = 'models/garbage_classifier_v2_earlystop.h5'
CLASS_NAMES_PATH = 'models/class_names.pkl'
IMAGE_SIZE = (224, 224)
CONFIDENCE_THRESHOLD = 75.0 

try:
    model = tf.keras.models.load_model(MODEL_PATH)
    with open(CLASS_NAMES_PATH, 'rb') as f:
        class_names = pickle.load(f)
except Exception as e:
    print(f"Error: Tidak bisa memuat model atau class names: {e}")
    exit()

recyclability_map = {
    'battery': ('Non-Recyclable', (0, 0, 255)),
    'biological': ('Non-Recyclable', (0, 0, 255)),
    'brown-glass': ('Recyclable', (0, 255, 0)),
    'cardboard': ('Recyclable', (0, 255, 0)),
    'clothes': ('Recyclable', (0, 255, 0)),
    'green-glass': ('Recyclable', (0, 255, 0)),
    'metal': ('Recyclable', (0, 255, 0)),
    'paper': ('Recyclable', (0, 255, 0)),
    'plastic': ('Recyclable', (0, 255, 0)),
    'shoes': ('Non-Recyclable', (0, 0, 255)),
    'trash': ('Non-Recyclable', (0, 0, 255)),
    'white-glass': ('Recyclable', (0, 255, 0))
}

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Kamera Error")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)

    h, w, _ = frame.shape
    box_size = 350
    x1 = (w - box_size) // 2
    y1 = (h - box_size) // 2
    x2 = x1 + box_size
    y2 = y1 + box_size
    roi = frame[y1:y2, x1:x2]

    img_resized = cv2.resize(roi, IMAGE_SIZE)
    img_array = np.expand_dims(img_resized, axis=0)

    prediction = model.predict(img_array, verbose=0)
    confidence = np.max(prediction[0]) * 100
    
    box_color = (0, 255, 255)
    
    if confidence > CONFIDENCE_THRESHOLD:
        predicted_class_index = np.argmax(prediction[0])
        predicted_class_name = class_names[predicted_class_index]
        recyclable_status, status_color = recyclability_map.get(predicted_class_name, ("N/A", (255, 255, 255)))
        box_color = status_color
        
        text_class = f"Jenis: {predicted_class_name}"
        text_confidence = f"Keyakinan: {confidence:.2f}%"
        text_status = f"Status: {recyclable_status}"
    else:
        text_class = "Arahkan sampah ke dalam kotak"
        text_confidence = ""
        text_status = ""


    text_y_position = y1 + 25 

    overlay = frame.copy()
    cv2.rectangle(overlay, (x1, y1), (x2, y1 + 100), (0, 0, 0), -1) 
    alpha = 0.6
    frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)
    
    cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 3)

    font_scale = 0.7
    font_thickness = 1
    
    cv2.putText(frame, text_class, (x1 + 15, text_y_position), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)
    cv2.putText(frame, text_confidence, (x1 + 15, text_y_position + 30), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)
    cv2.putText(frame, text_status, (x1 + 15, text_y_position + 60), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)

    cv2.imshow('Smart Garbage Classifier - Tekan "q" untuk keluar', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()